In [ ]:
import xarray as xr
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import gamma
from scipy.stats import genextreme as gev

from pyextremes import EVA

In [ ]:
# Load the ERA5 reanalysis daily precipitation data for Tenerife from 1950 - 2000 
file_path = 'data/input/ERA5_Tenerife_total_precipitation_day_1950_2000.nc'

daily_precipitation = xr.open_dataset(file_path)
# Add an attribute to the dataset
daily_precipitation.attrs['units'] = 'mm/d'
# create a array which only conains the years 
years = daily_precipitation['time'].dt.year.values

In [ ]:
# Print the loaded data base
daily_precipitation

# Select on grid cell accroding to coordinates 
latitude_target = 28.25
longitude_target = -16.76
daily_precipitation = daily_precipitation.sel(latitude=latitude_target, longitude=longitude_target, method='nearest')

In [ ]:
# Calculate monthly sums
monthly_sum_precipitation = daily_precipitation.resample(time='1m').sum(dim='time')
# Calaculate a mean of monthly sums from the years
annual_mean_precipitation_monthly_sum = monthly_sum_precipitation.groupby('time.month').mean(dim='time')


In [ ]:
# Convert DataArray to pandas DataFrame for easier plotting
df = annual_mean_precipitation_monthly_sum.to_dataframe()

# Plot mean monthly precipitation of multi-annual means as a bar plot
df['tp'].plot(kind='bar', figsize=(10, 6))
plt.title('Spatial Mean of Multi-Annual Monthly Total Precipitation (Entire Domain)')
plt.xlabel('Month')
plt.ylabel('Total Precipitation [mm]')
plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
# A diffrent way of ploting the precipitation allows us to learn something
# about distribution of values within the months

# First calaculate spatial mean for daily values which are grouped by month
spatial_mean_daily_precipitation_monthly = daily_precipitation['tp'].groupby('time.month')

# Prepare data for violin plots (daily values must be stored in lists for every month)
data_list = []
data_99p_list = []
months = range(1, 13)  # Months range from 1 to 12
for m in months:
    # Extract data for each month
    data_month = spatial_mean_daily_precipitation_monthly[m]  
    # Calaculate 99% percentile for each month
    data_month_99p = data_month.reduce(np.percentile, q=99, dim='time')   
    # store data into lists for each month 
    data_list.append(data_month.values)
    data_99p_list.append(data_month_99p.values)
    
# Create violin plots we use the seaborn libary
plt.figure(figsize=(10, 6))
sns.violinplot(data=data_list, cut=0, palette="Blues")
plt.title('Multi-Annual Daily Precipitation Distribution per Month')
plt.xlabel('Month')
plt.ylabel('Precipitation [mm]')
plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.grid(axis='y')

plt.plot(np.arange(0,12,1),data_99p_list, color='r', linestyle='--', label=f'99th Percentile')
plt.show()

In [ ]:
# Optain array of precipitation data from the xarray
daily_mean = daily_precipitation['tp']

# filter out only wet days (daily precipitation > 1mm)
daily_mean_wet = daily_mean[(daily_mean > 1)]
daily_mean_dry = daily_mean[(daily_mean <= 1)]

n_days = daily_mean.size
n_wet_days = daily_mean_wet.size
n_dry_days = daily_mean_dry.size

p_wet_day = n_wet_days/n_days

print(f'In total there are {n_wet_days:.0f} wet days out of {n_days:.0f} observations')
print(f'The mean daily propabilty of wet day is {p_wet_day:.2f} dry days out of {n_days:.0f} observations')

In [ ]:
# Calculate mean, median, and percentiles
mean_precip = daily_mean_wet.mean().values
median_precip = np.nanmedian(daily_mean_wet.values)
percentile_95 = np.nanpercentile(daily_mean_wet.values, 95)
percentile_99 = np.nanpercentile(daily_mean_wet.values, 99)

In [ ]:
# Plot the histogram
plt.figure(figsize=(8, 6))
plt.hist(daily_mean_wet, bins=50, alpha=0.7, color='skyblue', edgecolor='black')

# Add lines for mean, median, and percentiles
plt.axvline(mean_precip, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_precip:.2f}')
plt.axvline(median_precip, color='green', linestyle='dashed', linewidth=1, label=f'Median: {median_precip:.2f}')
plt.axvline(percentile_95, color='orange', linestyle='dashed', linewidth=1, label=f'95th Percentile: {percentile_95:.2f}')
plt.axvline(percentile_99, color='purple', linestyle='dashed', linewidth=1, label=f'99th Percentile: {percentile_99:.2f}')

# Plot legend, labels, and title
plt.legend()
plt.xlabel('Daily Mean Precipitation (mm)')
plt.ylabel('Frequency')
plt.title('Histogram of Daily Mean Precipitation with Percentiles')
plt.grid(True)
plt.show()

In [ ]:
# Create a DataFrame for better organization and visualization
df = pd.DataFrame({'Date': daily_mean_wet['time'], 'DailyRecords': daily_mean_wet.values})
df

Perform the following analysis based on the theoretical example from the lecture:

 - Fit a well-suited distribution function to the (non-zero) precipitation data.
 - Calculate the return period for events > 30 mm. Note that the model is based on wet days only, but that there is a large fraction of dry days, which affects the resulting annual probability.
 - Apply the block maximum method and calculate the return period for events > 30 mm.
 